<a href="https://colab.research.google.com/github/a01137646/MNA_Public/blob/main/Inteligencia%20artificial%20y%20aprendizaje%20autom%C3%A1tico/MNA_IAyAA_semana_6_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 6**
###**Árboles de decisión y bosque aleatorio.**


**Nombres y matrículas de los integrantes del equipo:**

*   Gerardo Aaron Castañeda Jaramillo - A01137646
*   
*   
*   
*   


En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [36]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.metrics import confusion_matrix

from sklearn.datasets import make_classification

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score         

from sklearn.metrics import classification_report, make_scorer
from imblearn.metrics import geometric_mean_score, classification_report_imbalanced
from sklearn.model_selection import  cross_validate,  RepeatedStratifiedKFold 

#**Ejercicio-1.** 

Convertimos la base de datos a una dataframe

In [22]:
df = pd.read_csv('https://github.com/a01137646/MNA_Public/blob/main/Inteligencia%20artificial%20y%20aprendizaje%20autom%C3%A1tico/SouthGermanCredit.asc?raw=true',delimiter =" ")

Cambimos los nombres de las columnas a nombres en ingles

In [23]:
df.rename(columns = {'laufkont':'status', 'laufzeit':'duration', 'moral':'credit_history', 'verw':'purpose', 'hoehe':'amount', 'sparkont':'savings', 'beszeit':'employment_duration', 
                     'rate':'installment_rate', 'famges':'personal_status_sex', 'buerge':'other_debtors', 'wohnzeit':'present_residence', 'verm':'property', 'alter':'age', 
                     'weitkred':'other_installment_plans', 'wohn':'housing', 'bishkred':'number_credits', 'beruf':'job', 'pers':'people_liable', 'telef':'telephone', 'gastarb':'foreign_worker',
                     'kredit':'credit_risk'}, inplace = True)
df.head()

,status,duration,credit_history,purpose,amount,savings,employment_duration,installment_rate,personal_status_sex,other_debtors,...,property,age,other_installment_plans,housing,number_credits,job,people_liable,telephone,foreign_worker,credit_risk
0,1,18,4,2,1049,1,2,4,2,1,...,2,21,3,1,1,3,2,1,2,1
1,1,9,4,0,2799,1,3,2,3,1,...,1,36,3,1,2,3,1,1,2,1
2,2,12,2,9,841,2,4,2,2,1,...,1,23,3,1,1,2,2,1,2,1
3,1,12,4,0,2122,1,3,3,3,1,...,1,39,3,1,2,2,1,1,1,1
4,1,12,4,0,2171,1,3,4,3,1,...,2,38,1,2,2,2,2,1,1,1


Vemos que no existan valores nulos o de tipo object

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   status                   1000 non-null   int64
 1   duration                 1000 non-null   int64
 2   credit_history           1000 non-null   int64
 3   purpose                  1000 non-null   int64
 4   amount                   1000 non-null   int64
 5   savings                  1000 non-null   int64
 6   employment_duration      1000 non-null   int64
 7   installment_rate         1000 non-null   int64
 8   personal_status_sex      1000 non-null   int64
 9   other_debtors            1000 non-null   int64
 10  present_residence        1000 non-null   int64
 11  property                 1000 non-null   int64
 12  age                      1000 non-null   int64
 13  other_installment_plans  1000 non-null   int64
 14  housing                  1000 non-null   int64
 15  numbe

#**Ejercicio-2.**

Separamos los valores X de los valores Y

In [12]:
Y = df['credit_risk']
X = df.drop('credit_risk',axis=1)

Hacemos una particion de los valores, 85% para entrenamiento, y 15% para pruebas

In [15]:
Xtv, Xtest, ytv, ytest = train_test_split(X, Y, train_size=0.85, test_size=0.15, random_state=1)  
print(Xtv.shape, ': dimensión de datos de entrada de entrenamiento y validación')
print(Xtest.shape, ': dimensión de datos de entrada de prueba')  
print(ytv.shape, ': dimensión de variable de salida para entrenamiento y validación')
print(ytest.shape, ': dimensión de variable de salida para prueba')

(850, 20) : dimensión de datos de entrada de entrenamiento y validación
(150, 20) : dimensión de datos de entrada de prueba
(850,) : dimensión de variable de salida para entrenamiento y validación
(150,) : dimensión de variable de salida para prueba


#**Ejercicio-3.**

Creamos listas con los nombres de las columnas dependiendo de su tipo de datos

In [25]:
#valores numericos
num_nombres = ['duration','amount','age']
#valores categoricos
cat_nombres = ['status','credit_history','purpose','savings','personal_status_sex','other_debtors','other_installment_plans','housing']
#valores ordinales
ord_nombres = ['employment_duration','installment_rate','present_residence','property','number_credits','job']
#valores binarios
bin_nombres = ['people_liable','telephone','foreign_worker']

Observamos si existen valores nulos que necesiten imputacion

In [26]:
df.isnull().values.any()

False

Aunque no existen valores nulos, por fines del ejercicio, los trabajaremos como si existieran.

Utilizamos el metodo de imputar por la mediana en los valores numericos, asi como el metodo de imputar la moda para los valores categoricos. 

De igual manera, para los valores numericos, se utiliza el metodo de MinMaxScaler para transformarlos a un rango entre 0 y 1. Mientras que para los valores categoricos y binarios se utiliza one-hot encoder, lo cual los transforma en los categoricos a variables dummies con valores de 0 y 1, y las binarias de igual manera, a valores de 0 y 1. 

In [27]:
# Transformaciones a factores numéricos de entrada:
num_pipeline = Pipeline(steps = [('impMediana', SimpleImputer(strategy='median')),
                                 ('escalaNum', MinMaxScaler())]) 
num_pipeline_nombres = num_nombres

# Transformaciones a factores categóricos de entrada:
catImp_pipeline = Pipeline(steps = [('impModa', SimpleImputer(strategy='most_frequent'))])  
catImp_pipeline_nombres = cat_nombres

# Transformaciones one-hot-encoder en algunas de las categóricas:
catOHE_pipeline = Pipeline(steps = [('OneHotE', OneHotEncoder())])
catOHE_pipeline_nombres = cat_nombres + bin_nombres

# Conjuntamos las transformaciones que se estarán aplicando a los datos de entrada:
columnasTransformer = ColumnTransformer(transformers = [('numpipe', num_pipeline, num_pipeline_nombres),
                                                        ('catimp', catImp_pipeline, catImp_pipeline_nombres),
                                                        ('catohe', catOHE_pipeline, catOHE_pipeline_nombres)],
                                        remainder='passthrough')

#**Ejercicio-4.**

In [38]:
def get_models():
  modelos = list()
  nombres = list()

  # LR - Regresión Logística:
  modelos.append(LogisticRegression(solver='liblinear'))
  nombres.append('LR')

  # DT - Árbol de Decisión:
  modelos.append(DecisionTreeClassifier())
  nombres.append('DT')
  
  # RF - Bosque Aleatorio:
  modelos.append(RandomForestClassifier())
  nombres.append('RF')

  return modelos, nombres

In [41]:
def mi_accuracy(yreal, ypred):
  tn, fp, fn, tp = confusion_matrix(yreal, ypred).ravel() 
  return (tp + tn)/(tp + tn + fp + fn)

def mi_precision(yreal, ypred):
  tn, fp, fn, tp = confusion_matrix(yreal, ypred).ravel() 
  return tp/(tp+fp)

def mi_recall(yreal, ypred):
  tn, fp, fn, tp = confusion_matrix(yreal, ypred).ravel() 
  return tp/(tp+fn)

def mi_f1score(yreal, ypred):
  tn, fp, fn, tp = confusion_matrix(yreal, ypred).ravel() 
  return tp/(tp + (1/2)*(fp + fn))

def mi_gmean(yreal, ypred):
  tn, fp, fn, tp = confusion_matrix(yreal, ypred).ravel()
  recall = tp/(tp+fn)
  specificity = tn/(tn+fp)
  return np.sqrt(recall*specificity)

In [42]:
modelos, nombres = get_models()  # cargamos los modelos a comparar
resultados = list()   

for i in range(len(modelos)):

  pipeline = Pipeline(steps=[('ct',columnasTransformer),('m',modelos[i])])
  cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=0)

  mismetricas = {'accuracy':make_scorer(mi_accuracy), 'precision':make_scorer(mi_precision), 'recall':make_scorer(mi_recall), 'f1score':make_scorer(mi_f1score), 'gmean':make_scorer(mi_gmean)}

  resultadosOU = cross_validate(pipeline, Xtv, ytv, scoring=mismetricas, cv=cv)
  resultados.append(resultadosOU)



  print('%s:\nmean Accuracy: %.3f (%.4f)\nmean Precision: %.3f (%.4f)\nmean Recall: %.3f (%.4f)\nmean f1score: %.3f (%.4f)\nGmean: %.3f (%.4f)\n' % (nombres[i],
                                                                          np.mean(resultadosOU['test_accuracy']),
                                                                          np.std(resultadosOU['test_accuracy']), 
                                                                          np.mean(resultadosOU['test_precision']),
                                                                          np.std(resultadosOU['test_precision']),
                                                                          np.mean(resultadosOU['test_recall']),
                                                                          np.std(resultadosOU['test_recall']),
                                                                          np.mean(resultadosOU['test_f1score']),
                                                                          np.std(resultadosOU['test_f1score']),
                                                                          np.mean(resultadosOU['test_gmean']),
                                                                          np.std(resultadosOU['test_gmean']),
                                                                          ))

LR:
mean Accuracy: 0.759 (0.0268)
mean Precision: 0.800 (0.0222)
mean Recall: 0.873 (0.0292)
mean f1score: 0.835 (0.0186)
Gmean: 0.657 (0.0466)

DT:
mean Accuracy: 0.688 (0.0290)
mean Precision: 0.785 (0.0251)
mean Recall: 0.762 (0.0456)
mean f1score: 0.772 (0.0253)
Gmean: 0.624 (0.0474)

RF:
mean Accuracy: 0.752 (0.0208)
mean Precision: 0.790 (0.0144)
mean Recall: 0.876 (0.0278)
mean f1score: 0.831 (0.0155)
Gmean: 0.638 (0.0300)



#**Ejercicio-5.**

In [ ]:
None

#**Ejercicio-6.**

In [ ]:
None

###**Fin de la Actividad de la semana 6.**